In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-for-llm/wizard_of_oz.txt


In [48]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 100000
learning_rate = 3e-4
eval_iters =1000

cuda


In [49]:
with open('/kaggle/input/text-for-llm/wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
chars=sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [50]:
#character to integer encode and decode
#character level tokenizer 
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [string_to_int[c] for c in s]
decode=lambda l: ''.join([int_to_string[i] for i in l])
data=torch.tensor(encode(text), dtype=torch.long)


In [51]:
n=int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device),y.to(device)
    return x,y
    
x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[76, 58, 71, 58,  1, 67, 68, 73],
        [74, 67, 60,  1, 60, 62, 71, 65],
        [58,  1, 47, 62, 79, 54, 71, 57],
        [66, 54, 67,  9,  1, 72, 58, 71]], device='cuda:0')
targets:
tensor([[58, 71, 58,  1, 67, 68, 73,  1],
        [67, 60,  1, 60, 62, 71, 65,  1],
        [ 1, 47, 62, 79, 54, 71, 57, 23],
        [54, 67,  9,  1, 72, 58, 71, 62]], device='cuda:0')


In [52]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [53]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1) 
            index_next = torch.multinomial(probs, num_samples=1) 
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)




Err7 vBaFWK[tzSKvqH'7sp6bcmMT;Vg7qd]x;V&NcF';V4i7﻿B!80B'0R0Huv2JWjIO[sI',;EX85g)04sH!'XUy&DT"Wq-EG,oO6aTGYp.2ir99_'W*0'ah!USN":*VdgWJZwk9O)xGDM*MgIBq4﻿_7ob,xbgaHV_I3Zy
GfrgN_[q9_QgdY﻿[3C'8hgGURor4uJQC&1n;DhVK.uCjR._buWIw&xO8.s;V(zV9C*s)(JZwJ3Hy44J?c:QC 4:P
Xcmp_nKXUcb.EK2vf8NpQ3C8W o.2R﻿waFM-QZWhRzw;VK9p,EFcS_lIA4:1I)LFjRmpx6RDTmaF!b3IBH5d5l_Gn,IMGeP
w"W_'qM9:l6I:E85wSfi5OI33:&XDRVjg8hmPbgGkXUB.WhUaz
o(Zz_R5KPDg81lWGjWzwhvpmD:2kX'5R"yIM!,8hi er6:GjgG﻿X(v1xdQmSyiSpQsz﻿l'fzwi2urBHW_HRUAnPUkolI3zwE


In [54]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
        
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 5.010, val loss: 5.018
step: 1000, train loss: 4.767, val loss: 4.764
step: 2000, train loss: 4.540, val loss: 4.541
step: 3000, train loss: 4.321, val loss: 4.317
step: 4000, train loss: 4.128, val loss: 4.133
step: 5000, train loss: 3.937, val loss: 3.943
step: 6000, train loss: 3.781, val loss: 3.795
step: 7000, train loss: 3.643, val loss: 3.638
step: 8000, train loss: 3.495, val loss: 3.501
step: 9000, train loss: 3.376, val loss: 3.392
step: 10000, train loss: 3.255, val loss: 3.272
step: 11000, train loss: 3.174, val loss: 3.195
step: 12000, train loss: 3.079, val loss: 3.104
step: 13000, train loss: 2.997, val loss: 3.022
step: 14000, train loss: 2.928, val loss: 2.948
step: 15000, train loss: 2.882, val loss: 2.889
step: 16000, train loss: 2.825, val loss: 2.846
step: 17000, train loss: 2.782, val loss: 2.794
step: 18000, train loss: 2.735, val loss: 2.773
step: 19000, train loss: 2.719, val loss: 2.732
step: 20000, train loss: 2.678, val loss: 2.708
step:

In [55]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"Wild an hekincres thicthas unca oin the wid curontizand waled-TEman'tissn aw, te s be o b, s, til thinsacomug g hith igimy-Andvecetit ate bupand the!*Q_by.
" thtenene eoustht sed ewomure d

Ze hee ve crtase, Jisth,"


s whee went a deshe Gay, t ha ulotoreo velyoas t gres oplo. se ff
" ors apordrs boimass.
" cuth uikeanl thesomiveron't sshexpreang f s um t veghance "Bu m-oun, ty Dourrannorowembath t band t. h ge bry-b.

"So as, thouran

Thor ir

wire hesu hene. wof whace bas'muttter
"
sestingsth
